In [1]:
# this note book extracts out the statistics from landsat fractional cover imagery
# it first generates a list of all avilable fc imagery and then matchs them to individual field sites based on a given
# day range around the field site measured date.

# the stats are then extracted out of the imagery within the day range given for each site to produce a single csv file 
# the outputs can then be used to assess the accuracy of the fractional cover data. 

In [4]:
import pandas as pd
import geopandas as gpd
import fnmatch
import os
import datetime
from datetime import timedelta
import glob
import pdb
import sys
import argparse
import csv
import subprocess
import rasterio
import numpy as np
import shutil

In [5]:
# produce an updated list of available imagery form the shared satellite drive.

%run C:/Users/grants/code/file_man/list_of_files_multi_dir_fnmatch.py -d Z:/Landsat/wrs2/ -e *dilm[2-4]_zstdmask.img -o imglist_dil.csv

Z:/Landsat/wrs2/099_078\2015\201501\l7tmre_p099r078_20150126_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201501\l8olre_p099r078_20150102_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201501\l8olre_p099r078_20150118_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201502\l7tmre_p099r078_20150227_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201502\l8olre_p099r078_20150203_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201502\l8olre_p099r078_20150219_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201503\l7tmre_p099r078_20150315_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201503\l7tmre_p099r078_20150331_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201503\l8olre_p099r078_20150323_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201504\l7tmre_p099r078_20150416_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201504\l8olre_p099r078_20150408_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201504\l8olre_p099r078_20150424_dilm4_zstdmask.img
Z:/Landsat/wrs2/099_078\2015\201505\l7tm

Z:/Landsat/wrs2/100_075\1998\199807\l5tmre_p100r075_19980705_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199807\l5tmre_p100r075_19980721_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199808\l5tmre_p100r075_19980806_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199808\l5tmre_p100r075_19980822_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199809\l5tmre_p100r075_19980907_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199810\l5tmre_p100r075_19981025_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199811\l5tmre_p100r075_19981126_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1998\199812\l5tmre_p100r075_19981212_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1999\199901\l5tmre_p100r075_19990113_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1999\199901\l5tmre_p100r075_19990129_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1999\199902\l5tmre_p100r075_19990214_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1999\199903\l5tmre_p100r075_19990302_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\1999\199903\l5tm

Z:/Landsat/wrs2/100_075\2008\200802\l5tmre_p100r075_20080207_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200802\l5tmre_p100r075_20080223_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200802\l7tmre_p100r075_20080215_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200803\l5tmre_p100r075_20080310_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200803\l5tmre_p100r075_20080326_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200803\l7tmre_p100r075_20080302_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200803\l7tmre_p100r075_20080318_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200804\l5tmre_p100r075_20080411_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200804\l5tmre_p100r075_20080427_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200804\l7tmre_p100r075_20080403_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200804\l7tmre_p100r075_20080419_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200805\l7tmre_p100r075_20080505_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2008\200805\l7tm

Z:/Landsat/wrs2/100_075\2018\201807\l8olre_p100r075_20180728_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201808\l7tmre_p100r075_20180805_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201808\l7tmre_p100r075_20180821_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201808\l8olre_p100r075_20180813_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201808\l8olre_p100r075_20180829_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201809\l7tmre_p100r075_20180906_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201809\l7tmre_p100r075_20180922_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201809\l8olre_p100r075_20180914_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201809\l8olre_p100r075_20180930_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201810\l7tmre_p100r075_20181008_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201810\l7tmre_p100r075_20181024_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201810\l8olre_p100r075_20181016_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_075\2018\201811\l7tm

Z:/Landsat/wrs2/100_076\1992\199212\l5tmre_p100r076_19921211_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1992\199212\l5tmre_p100r076_19921227_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199301\l5tmre_p100r076_19930112_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199301\l5tmre_p100r076_19930128_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199302\l5tmre_p100r076_19930213_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199303\l5tmre_p100r076_19930301_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199304\l5tmre_p100r076_19930402_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199304\l5tmre_p100r076_19930418_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199305\l5tmre_p100r076_19930520_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199306\l5tmre_p100r076_19930621_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199307\l5tmre_p100r076_19930723_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199308\l5tmre_p100r076_19930808_dilm3_zstdmask.img
Z:/Landsat/wrs2/100_076\1993\199308\l5tm

...

Z:/Landsat/wrs2/106_073\1990\199005\l5tmre_p106r073_19900506_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199005\l5tmre_p106r073_19900522_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199006\l5tmre_p106r073_19900607_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199007\l5tmre_p106r073_19900709_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199007\l5tmre_p106r073_19900725_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199008\l5tmre_p106r073_19900810_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199008\l5tmre_p106r073_19900826_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199009\l5tmre_p106r073_19900911_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199009\l5tmre_p106r073_19900927_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199010\l5tmre_p106r073_19901013_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199010\l5tmre_p106r073_19901029_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199011\l5tmre_p106r073_19901114_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\1990\199012\l5tm

Z:/Landsat/wrs2/106_073\2000\200012\l7tmre_p106r073_20001203_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2000\200012\l7tmre_p106r073_20001219_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200101\l7tmre_p106r073_20010104_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200101\l7tmre_p106r073_20010120_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200102\l7tmre_p106r073_20010205_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200102\l7tmre_p106r073_20010221_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200103\l7tmre_p106r073_20010309_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200103\l7tmre_p106r073_20010325_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200104\l7tmre_p106r073_20010410_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200104\l7tmre_p106r073_20010426_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200105\l7tmre_p106r073_20010512_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200105\l7tmre_p106r073_20010528_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2001\200106\l7tm

Z:/Landsat/wrs2/106_073\2009\200908\l5tmre_p106r073_20090830_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200908\l7tmre_p106r073_20090806_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200908\l7tmre_p106r073_20090822_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200909\l5tmre_p106r073_20090915_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200909\l7tmre_p106r073_20090907_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200909\l7tmre_p106r073_20090923_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200910\l5tmre_p106r073_20091001_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200910\l5tmre_p106r073_20091017_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200910\l7tmre_p106r073_20091009_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200910\l7tmre_p106r073_20091025_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200911\l5tmre_p106r073_20091102_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200911\l7tmre_p106r073_20091110_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2009\200911\l7tm

Z:/Landsat/wrs2/106_073\2014\201405\l7tmre_p106r073_20140516_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201405\l8olre_p106r073_20140508_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201405\l8olre_p106r073_20140524_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201406\l7tmre_p106r073_20140601_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201406\l7tmre_p106r073_20140617_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201406\l8olre_p106r073_20140609_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201406\l8olre_p106r073_20140625_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201407\l7tmre_p106r073_20140703_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201407\l7tmre_p106r073_20140719_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201407\l8olre_p106r073_20140711_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201407\l8olre_p106r073_20140727_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201408\l7tmre_p106r073_20140804_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2014\201408\l7tm

Z:/Landsat/wrs2/106_073\2018\201808\l7tmre_p106r073_20180815_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201808\l7tmre_p106r073_20180831_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201808\l8olre_p106r073_20180807_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201808\l8olre_p106r073_20180823_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201809\l7tmre_p106r073_20180916_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201809\l8olre_p106r073_20180908_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201809\l8olre_p106r073_20180924_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201810\l7tmre_p106r073_20181002_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201810\l7tmre_p106r073_20181018_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201810\l8olre_p106r073_20181010_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201810\l8olre_p106r073_20181026_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201811\l7tmre_p106r073_20181103_dilm2_zstdmask.img
Z:/Landsat/wrs2/106_073\2018\201811\l7tm

In [6]:
df = pd.read_csv('imglist_dil.csv',header=None)
print(df.shape)
# get the path row and image date from the image file name
df['path_row'] = df[0].map(lambda x: str(x)[-35:-32]) + '_' + df[0].map(lambda x: str(x)[-31:-28])
df['img_date'] = df[0].map(lambda x: str(x)[-27:-23]) + '-' + df[0].map(lambda x: str(x)[-23:-21]) + '-' + df[0].map(lambda x: str(x)[-21:-19])
df['zone'] = df[0].map(lambda x: str(x)[-14:-13])
print (df['zone'].unique())
df.to_csv('imglist_dil_datetime_zone.csv')

(42071, 1)
['4' '3' '2']


In [8]:
import pandas as pd
import geopandas as gpd
import fnmatch
import os
import datetime
from datetime import timedelta
import glob
import pdb
import sys
import argparse
import csv
import subprocess
import rasterio
import numpy as np
import shutil



# make some temp dir's to put single site shp file and results into 

tempDir = './temp_individual_results'
tempshp = './temp_individual_shp'
    
# check if the temp dir exists and if it does remove it, otherwise create the new dir for the individual outputs
check_if_dir_exists = os.path.isdir(tempDir)
if check_if_dir_exists == True:
    shutil.rmtree(tempDir)
    os.makedirs(tempDir)
else:    
    os.makedirs(tempDir)
    
    
check_if_dir_exists = os.path.isdir(tempshp)
if check_if_dir_exists == True:
    shutil.rmtree(tempshp)
    os.makedirs(tempshp)
else:    
    os.makedirs(tempshp)

# read in the shape file and produce a date_time 
sd = gpd.read_file('C:/Users/grants/code/cal_val/field_shapefile/nt_rm_fieldSite_2021_wrs2sj_buff_test_double_wrs2.shp')
sd['date_time'] = pd.to_datetime(sd['Date'] ,yearfirst=False, dayfirst=True)
print (sd['date_time'])

# define the number of days either side of the field site measured date to extract stats from the available imagery 
number_of_days = 15

date_plus = number_of_days 
date_minus = -abs(number_of_days)
print (date_plus,date_minus)

# create the plus and minus date range 
sd["fwd_date"] = sd["date_time"] + timedelta(days=date_plus)
sd["bck_date"] = sd["date_time"] + timedelta(days=date_minus)


# generate a list of all the avilable imagery to extract the statstics from 





# iterrate over the sites in the shapefile and find the image matches and extract out the statistics 

for index, row in sd.iterrows():
    
    siteN = row['uid']
    
    sda = sd[(sd['uid'] == siteN )]
    print (sda['Site'])

    
    site = sda['Site']
    field_date = sda['date_time']
    print (field_date)
    # get the date range for the imagery stats requred
    search_date_plus =  row['fwd_date']
    search_date_minus = row['bck_date']                   
    print (search_date_plus) 
    print ('-------------------------------------------')
    # get the info to create the dir path to get a list of imagery 
    path_row = str(row['PATH']) + '_0' + str(row['ROW'])
    print (path_row)
    #search the img list and find matches
    df = pd.read_csv('imglist_dil_datetime_zone.csv',header=0)
    # make the img_date a time date column 
    df['img_dt'] = pd.to_datetime(df['img_date'], yearfirst=True, dayfirst=False) 
        
    dfs = df[(df['path_row'] == path_row)]
    print (dfs)
    imgS = dfs[dfs["img_dt"].isin(pd.date_range(search_date_minus,search_date_plus))]
    print (imgS)
    # get the zone from the imagery to inform the selection of the EPSG code to use.
    zone = imgS['zone'].unique()
    print (zone)
    
    # reproject the site polygon the same cordinate system of the imagery
    if zone == 2:
        
        sdsr = sda.to_crs("EPSG:32752")      
        print ('this is the projection ',sdsr.crs)
        print (list(sdsr))

    if zone == 3:
        sdsr = sda.to_crs("EPSG:32753")      
        print ('this is the projection ',sdsr.crs)
        print (sdsr)
        
    if zone == 4:
        sdsr.to_crs("EPSG:32754")      
        print ('this is the projection ',sdsr.crs)
        print (sdsr)
    
    temp_shp = sdsr[['uid', 'Station', 'Site', 'Date', 'C_Lat', 'C_Lon', 'NO_Lat', 'NO_Lon', 'ba_trees','ba_shrubs', 'ba_total', 'FPC', 'PPC', 'CC', 'PVg', 'NPVg', 'BGg', 'PV', 'NPV', 'BG','index_righ', 'AREA', 'PERIMETER', 'PR_', 'PR_ID', 'RINGS_OK', 'RINGS_NOK', 'PATH','ROW', 'MODE', 'SEQUENCE', 'WRSPR', 'PR', 'ACQDayL7', 'ACQDayL8', 'geometry']]
    
    
    
    temp_shp.to_file('./temp_individual_shp/temp_' + str(siteN) + '_.shp')
    
    shp_file = './temp_individual_shp/temp_' + str(siteN) + '_.shp'
    
    print ('Site name:',site)
    print ('field date:',field_date)
    print (path_row)
    print ('start search date:',search_date_minus) 
    for index, row in imgS.iterrows():
        print (row['0'])
    print ('end search date:',search_date_plus)

    #uid = uid
    for index, row in imgS.iterrows():
        
        site_name = str(site)
        
        date = str(field_date) 
        
        img = str(row['0'])
        print (img)
        
        nodata = 0 
        
        #uid = 'uid'
        
        shp = shp_file
        #print (shp)
        
        csv = './temp_individual_results/results_' + str(siteN) + '_' + str(index) + '.csv'
        
        #pdb.set_trace()
        
        print (csv)
        
        
        cmd = "python zonal_stats_single_cal_val_local.py --image %s --nodata %s --shape %s --csv %s"% (img,nodata,shp, csv) 
       
        os.system(cmd)
        #print (sdsr.crs)
        #print (sdsr)

        print ("...................")
        
   


    # select out the individual site and reproject it to projected coords UTM 52 or 53 and extract out the stats from 
    # the given img_selection list using the script - zonal_stats_single_perc.py
    
    # use geopandas to create the shp file in memory - may need to write it out as a file? if so make it temp and delete...
    
    # create a temp dir to save the individual results before concating them together 
  
   
       
# read in the individual band results and concatenate them to a single dataframe
tempRes = "./temp_individual_results/"

all_files = glob.glob(os.path.join(tempRes, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
    
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file,ignore_index=False, axis=0)

# export the results to a csv file
concatenated_df.to_csv('cal_val_dil_data_2021_results30days_test2.csv')     


0   2021-09-04
1   2021-08-31
2   2021-09-04
3   2021-08-31
Name: date_time, dtype: datetime64[ns]
15 -15
0    BDA14A
2    BDA14A
Name: Site, dtype: object
0   2021-09-04
2   2021-09-04
Name: date_time, dtype: datetime64[ns]
2021-09-19 00:00:00
-------------------------------------------
106_073
       Unnamed: 0                                                  0 path_row  \
41080       41080  Z:/Landsat/wrs2/106_073\1988\198801\l5tmre_p10...  106_073   
41081       41081  Z:/Landsat/wrs2/106_073\1988\198801\l5tmre_p10...  106_073   
41082       41082  Z:/Landsat/wrs2/106_073\1988\198802\l5tmre_p10...  106_073   
41083       41083  Z:/Landsat/wrs2/106_073\1988\198802\l5tmre_p10...  106_073   
41084       41084  Z:/Landsat/wrs2/106_073\1988\198803\l5tmre_p10...  106_073   
...           ...                                                ...      ...   
42066       42066  Z:/Landsat/wrs2/106_073\2021\202108\l8olre_p10...  106_073   
42067       42067  Z:/Landsat/wrs2/106_073\2021\202109\

...................
Z:/Landsat/wrs2/106_072\2021\202109\l7tmre_p106r072_20210908_dilm2_zstdmask.img
./temp_individual_results/results_24_41076.csv
...................
Z:/Landsat/wrs2/106_072\2021\202109\l8olre_p106r072_20210916_dilm2_zstdmask.img
./temp_individual_results/results_24_41078.csv
...................
1    BDA15A
3    BDA15A
Name: Site, dtype: object
1   2021-08-31
3   2021-08-31
Name: date_time, dtype: datetime64[ns]
2021-09-15 00:00:00
-------------------------------------------
106_072
       Unnamed: 0                                                  0 path_row  \
40107       40107  Z:/Landsat/wrs2/106_072\1988\198801\l5tmre_p10...  106_072   
40108       40108  Z:/Landsat/wrs2/106_072\1988\198801\l5tmre_p10...  106_072   
40109       40109  Z:/Landsat/wrs2/106_072\1988\198802\l5tmre_p10...  106_072   
40110       40110  Z:/Landsat/wrs2/106_072\1988\198802\l5tmre_p10...  106_072   
40111       40111  Z:/Landsat/wrs2/106_072\1988\198803\l5tmre_p10...  106_072   
...      